In [1]:
from config import *
import requests
import pandas as pd

In [2]:
def get_dates(start, end):
    return pd.Series(pd.date_range(start, end,freq='d').format()) # Format makes the output ans str variable.

get_dates('1992-01-01','1992-01-03')

0    1992-01-01
1    1992-01-02
2    1992-01-03
dtype: object

In [3]:
def city_code(city_name):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/US/USD/en-US/"

    params = {"query": city_name}

    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
                      'x-rapidapi-key': "5f8cfd5afcmshd4d1c1f353a0a37p1a0c57jsnc156b5523fcf"}

    response = requests.get(url, headers = headers, params = params)

    return response.json()["Places"][0]["PlaceId"]

In [8]:
def flight_prices(departure, arrival, date):

    departure_code = city_code(departure)
    arrival_code = city_code(arrival)
    url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/{departure_code}/{arrival_code}/{date}"

    params = {"inboundpartialdate":{date}}

    #headers = {
    #'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    #'x-rapidapi-key': "<introduce your RapidAPI key here>"}

    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': "7a691558dfmsh23dd1510be0dce8p1bc166jsne881b5935099"}

    response = requests.get(url, headers=headers, params=params)

    return response.json()

flight_prices('Paris', 'London','2021-12-10')

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
from  time import sleep

def flight_days(origin, destination, start, end):
    dates = get_dates(start, end)
    my_dict = {}
    for date in list(dates):
        #result = None
        #while (result == None):
            try: 
                #print(date)
                #print("Gathering fligths")
                result = flight_prices(origin, destination, date)
                my_dict[date] = result
                sleep(5)
            except: 
                print("Error from the API")
                sleep(60)
    return my_dict

In [ ]:
flights = flight_days('Barcelona','London','2021-11-26','2021-12-12')

In [7]:
flights

{}

In [ ]:
def repeat(x): 
    _size = len(x) 
    repeated = [] 
    for i in range(_size): 
        k = i + 1
        for j in range(k, _size): 
            if x[i] == x[j] and x[i] not in repeated: 
                repeated.append(x[i]) 
    return repeated 

In [ ]:
import re
my_dict = {}
store_dict = []
store_price = []
for date in flights: # We iterate over the dates
    my_dict[date] = [] #We create an empty list of prices for the given date.
    min_price = 10000
    best_date = ""
    for elem in flights[date]['Quotes']: # We iterate over "Quotes" of each day.
        my_dict[date].append(elem['MinPrice']) # We fill the date with the prices.
        if (elem['MinPrice'] == min_price):
            min_price = elem['MinPrice']
            best_date = date
    try:
        store_dict.append([{'Price': flights[date]['Quotes'][0]['MinPrice'],
                           'Name origin': flights[date]['Places'][0]['Name'],
                           'Name arrival': flights[date]['Places'][1]['Name'],
                           'Airline': flights[date]['Carriers'][0]['Name'],
                           'Date': flights[date]['Quotes'][0]['OutboundLeg']['DepartureDate'][:-9]
                           }])
    except:
        continue

for elem in store_dict:
    store_price.append(elem[0]['Price'])
    
for i in store_dict:
    if i[0]['Price'] in repeat(store_price):
        print(i)